In [1]:
from lark import Lark
from lark.visitors import Transformer
from LabeledTransitionSystem import *

In [2]:
aut_parser = Lark.open("grammars/aut.lark", parser="earley")
mu_parser = Lark.open("grammars/mu.lark", parser="earley")

In [3]:
system = aut_parser.parse(open("../examples/test.aut").read())
formula = mu_parser.parse(open("../examples/test.mu").read())
print(system)

Tree(Token('RULE', 'start'), [Tree(Token('RULE', 'aut_header'), [Token('FIRST_STATE', '0'), Token('NR_OF_TRANSITIONS', '12'), Token('NR_OF_STATES', '10')]), Tree(Token('RULE', 'aut_edge'), [Token('START_STATE', '0'), Token('LABEL', '"lock(p2, f2)"'), Token('END_STATE', '1')]), Tree(Token('RULE', 'aut_edge'), [Token('START_STATE', '0'), Token('LABEL', '"lock(p1, f1)"'), Token('END_STATE', '2')]), Tree(Token('RULE', 'aut_edge'), [Token('START_STATE', '1'), Token('LABEL', '"lock(p1, f1)"'), Token('END_STATE', '3')]), Tree(Token('RULE', 'aut_edge'), [Token('START_STATE', '1'), Token('LABEL', '"lock(p2, f1)"'), Token('END_STATE', '4')]), Tree(Token('RULE', 'aut_edge'), [Token('START_STATE', '2'), Token('LABEL', '"lock(p2, f2)"'), Token('END_STATE', '3')]), Tree(Token('RULE', 'aut_edge'), [Token('START_STATE', '2'), Token('LABEL', '"lock(p1, f2)"'), Token('END_STATE', '5')]), Tree(Token('RULE', 'aut_edge'), [Token('START_STATE', '4'), Token('LABEL', '"eat(p2)"'), Token('END_STATE', '6')]), T

In [5]:
def merge(dicts):
    merged = {}
    for dict in dicts:
        for key, value in dict.items():
            if key in merged:
                merged[key] = merged[key].union(value)
            else:
                merged[key] = value
    return merged

class LTSTransformer(Transformer):
    def string(self, s):
        return s[1:-1]
    
    number = int

    FIRST_STATE = number
    NR_OF_TRANSITIONS = number
    NR_OF_STATES = number

    def aut_header(self, header):
        init_state, _, n_states = header
        return {"states": set(range(n_states)),
                "init_states": {init_state}}

    START_STATE = number
    LABEL = string
    END_STATE = number

    def aut_edge(self, transition):
        return {"transitions": {tuple(transition)}}

    def start(self, args):
        argdict = merge(args)
        argdict['action_labels'] = {al for (_, al, _) in argdict['transitions']}
        lts = LabeledTransitionSystem(**argdict)
        return lts

lts = LTSTransformer().transform(system)
%store lts

Stored 'lts' (LabeledTransitionSystem)
